In [1]:
# ==============================================================================
# V33 changes:
    # overview: V33 is built on the V32 codebase but intentionally reverts V31’s
    #   risky fine-tuning/augmentation choices back toward V30’s proven baseline.
    #   On top of that baseline, V33 adds targeted improvements (class-specific
    #   mild aug, targeted smoothing + focal, calibrated S1 threshold) to address
    #   persistent weaknesses in 'sadness' and 'speech_action' without reintroducing
    #   the V31 regressions.

    # section #4 - Stage 1 simplification:
    #   Reverted to a single, uniform learning rate (3e-5) managed by the Trainer.
    #   (Undoes V31’s discriminative LR / freezing complexity.)

    # section #4 - Augmentation scope correction:
    #   Removed 'sadness' and 'speech_action' from the heavy RandAugment list
    #   to avoid distorting subtle features (restores V30 behavior).

    # section #5a - Stage 2 targeted loss:
    #   Kept label smoothing at 0.05 overall, but turned OFF smoothing for
    #   ['sadness','speech_action'] and added mild focal scaling (gamma=1.5)
    #   inside TargetedSmoothedCrossEntropyLoss to emphasize hard examples.

    # section #6b - Stage 2 optimization stability:
    #   lr_scheduler_type="cosine", warmup_ratio=0.10, weight_decay=0.05,
    #   gradient_accumulation_steps=2, num_train_epochs=6, learning_rate=4e-5.

    # section #6c - Aug pipeline wiring:
    #   Introduced a *mild* augmentation pipeline specifically for
    #   ['sadness','speech_action'] and passed the MERGED `augment_dict`
    #   (minority_augment_map_s2 ∪ targeted_mild_map_s2) to the collator.

    # section #6d - Collator robustness (tensor-only erasing):
    #   Moved RandomErasing to a tensor-only step (ToTensor -> RandomErasing -> ToPIL)
    #   so torchvision sees a tensor (prevents 'Image' has no attribute 'shape').

    # section #6e - Fragile-class policy:
    #   Enabled per-class skip for RandomErasing via `skip_erasing_label_ids`
    #   for ['sadness','speech_action'] to preserve delicate cues, while
    #   keeping erasing active for other classes

    # section #6f - Minor collator naming fix:
    #   - Corrected attribute reference in DataCollatorWithAugmentation.__call__
    #     from self.base_augment_pil to self.base_augment to match the defined
    #     attribute name and prevent AttributeError.

    # section #7 - Stage 1 inference calibration:
    #   Switched hierarchical_predict to a probability threshold (tau=0.45)
    #   for 'relevant' instead of argmax to lift recall without retraining.

    # section #8 - Ensemble analysis demo:
    #   Made the demo runnable by auto-picking an image from any non-empty
    #   folder under 'review_candidates_by_predicted_class' with proper guards.

    # section #9 - Consistency cleanup:
    #   Standardized the processor call in hierarchical_ensemble_predict to
    #   use `images=image` for parity with the rest of the codebase.

    # section #10 - Operational defaults:
    #   Set RUN_INFERENCE=False and PREPARE_DATASETS=False by default to
    #   avoid heavy reprocessing during everyday runs. Toggle to True when needed.
# ==============================================================================

In [2]:
# --------------------------
# 0. Imports
# --------------------------
# WORKAROUND for PyTorch MPS bug
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

# Standard Library Imports
import datasets
import csv
import gc
import glob
import multiprocessing as mp
import os
import random
import re
import shutil
import subprocess
import sys
import time

# Third-Party Imports
import accelerate
import dill
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import transformers

# From Imports
from collections import Counter
from datasets import ClassLabel, Dataset, Features, Image as DatasetsImage, concatenate_datasets, load_dataset
from datetime import datetime
from functools import partial
from imagehash import phash, hex_to_hash
from io import BytesIO
from pathlib import Path
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.utils.class_weight import compute_class_weight
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW, LBFGS
from torchvision import transforms
from torchvision.transforms import (
    RandAugment,
)
from tqdm import tqdm
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
    ViTForImageClassification,
)

In [3]:
# --------------------------
# 1. Global Configurations
# --------------------------

# --- 📂 Core Paths ---
# This is the root directory containing your original 14-class dataset structure.
BASE_DATASET_PATH = "/Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfaga_dataset_14_labels"
# This is the root directory where all outputs (models, logs, prepared datasets) will be saved.
OUTPUT_ROOT_DIR = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training"

# --- ⚙️ Run Configuration ---
# default safer for daily dev runs; flip to True when you want full-corpus inference
RUN_INFERENCE = False
# default safer; run once when dataset layout changes
PREPARE_DATASETS = False

# Finds the most recent V* model directory based on modification time.
def find_latest_checkpoint(root_dir):
    all_run_dirs = [
        os.path.join(root_dir, d)
        for d in os.listdir(root_dir)
        if d.startswith("V") and os.path.isdir(os.path.join(root_dir, d))
    ]
    if not all_run_dirs:
        return None

    # Sort directories by modification time, newest first
    sorted_dirs = sorted(all_run_dirs, key=os.path.getmtime, reverse=True)

    # The newest directory is the current run's empty folder.
    # We need the second newest, which is the latest *completed* run.
    if len(sorted_dirs) > 1:
        return sorted_dirs[1] # <-- Return the second item in the list
    else:
        # If there's only one (or zero), no previous checkpoint exists
        return None

# --- 🤖 Model Configuration ---
# The pretrained Vision Transformer model from Hugging Face to be used as a base.
BASE_MODEL_NAME = "google/vit-base-patch16-224-in21k"

# Dynamically find the latest checkpoint to train from
latest_checkpoint = find_latest_checkpoint(OUTPUT_ROOT_DIR)

if latest_checkpoint:
    PRETRAINED_CHECKPOINT_PATH = latest_checkpoint
    print(f"✅ Dynamically loading latest checkpoint: {os.path.basename(PRETRAINED_CHECKPOINT_PATH)}")
else:
    # If no checkpoint is found, fall back to the base model from Hugging Face
    PRETRAINED_CHECKPOINT_PATH = BASE_MODEL_NAME
    print(f"⚠️ No local checkpoint found. Starting from base model: {BASE_MODEL_NAME}")
    
# --- 🏷️ Dataset & Label Definitions ---
# These lists define the structure for the hierarchical pipeline.
# All folders listed here will be grouped into the 'relevant' class for Stage 1
# and used for training the final 11-class classifier in Stage 2.
RELEVANT_CLASSES = [
    'anger', 'contempt', 'disgust', 'fear', 'happiness',
    'neutral', 'questioning', 'sadness', 'surprise',
    'neutral_speech', 'speech_action'
]
# **IMPORTANT**: Since 'unknown' is a subfolder of 'hard_case', we only need to
# list 'hard_case' here. The script will find all images inside it recursively.
IRRELEVANT_CLASSES = ['hard_case']

# Mappings for the Stage 2 (11-class Emotion) model
id2label_s2 = dict(enumerate(RELEVANT_CLASSES))
label2id_s2 = {v: k for k, v in id2label_s2.items()}

# Mappings for the Stage 1 (binary Relevance) model
id2label_s1 = {0: 'irrelevant', 1: 'relevant'}
label2id_s1 = {v: k for k, v in id2label_s1.items()}

# --- 🖼️ File Handling ---
# Defines valid image extensions and provides a function to check them.
VALID_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")
def is_valid_image(filename):
    return filename.lower().endswith(VALID_EXTENSIONS) and not filename.startswith("._")

# --- 🔢 Versioning and Output Directory Setup ---
# Automatically determines the next version number (e.g., V31) and creates a timestamped output folder.
def get_next_version(base_dir):
    all_entries = glob.glob(os.path.join(base_dir, "V*_*"))
    existing = [os.path.basename(d) for d in all_entries if os.path.isdir(d)]
    versions = [
        int(d[1:].split("_")[0]) for d in existing
        if d.startswith("V") and "_" in d and d[1:].split("_")[0].isdigit()
    ]
    next_version = max(versions, default=0) + 1
    return f"V{next_version}"

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION = get_next_version(OUTPUT_ROOT_DIR)
VERSION_TAG = VERSION + "_" + timestamp
SAVE_DIR = os.path.join(OUTPUT_ROOT_DIR, VERSION_TAG)
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Output directory created: {SAVE_DIR}")

✅ Dynamically loading latest checkpoint: V31_20251007_153512
📁 Output directory created: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V33_20251010_122749


In [4]:
# ----------------------------------------------------
# 2. Hierarchical Dataset Preparation
# ----------------------------------------------------
# This function organizes the original multi-class dataset into two separate
# folder structures required for the two-stage training process. It recursively
# searches through subdirectories (no matter how deep) and is smart enough to
# skip non-image files.
def prepare_hierarchical_datasets(base_path, output_path):
    
    stage1_path = os.path.join(output_path, "stage_1_relevance_dataset")
    stage2_path = os.path.join(output_path, "stage_2_emotion_dataset")

    print(f"🗂️ Preparing hierarchical datasets at: {output_path}")

    # --- Create Stage 1 Dataset (Relevance Filter) ---
    print("\n--- Creating Stage 1 Dataset ---")
    irrelevant_dest = os.path.join(stage1_path, "0_irrelevant")
    relevant_dest = os.path.join(stage1_path, "1_relevant")
    os.makedirs(irrelevant_dest, exist_ok=True)
    os.makedirs(relevant_dest, exist_ok=True)

    # Copy irrelevant files recursively
    print("Processing 'irrelevant' classes...")
    for class_name in IRRELEVANT_CLASSES:
        src_dir = Path(os.path.join(base_path, class_name))
        if src_dir.is_dir():
            print(f"  Recursively copying from '{class_name}'...")
            # Here, rglob('*') finds every file in every sub-folder.
            for file_path in src_dir.rglob('*'):
                if file_path.is_file() and is_valid_image(file_path.name):
                    shutil.copy(file_path, irrelevant_dest)
        else:
            print(f"  ⚠️ Warning: Source directory not found for '{class_name}'")

    # Copy relevant files recursively
    print("Processing 'relevant' classes...")
    for class_name in RELEVANT_CLASSES:
        src_dir = Path(os.path.join(base_path, class_name))
        if src_dir.is_dir():
            print(f"  Recursively copying from '{class_name}'...")
            for file_path in src_dir.rglob('*'):
                if file_path.is_file() and is_valid_image(file_path.name):
                    shutil.copy(file_path, relevant_dest)
        else:
            print(f"  ⚠️ Warning: Source directory not found for '{class_name}'")

    # --- Create Stage 2 Dataset (Emotion Classifier) ---
    print("\n--- Creating Stage 2 Dataset ---")
    for class_name in RELEVANT_CLASSES:
        src_dir = Path(os.path.join(base_path, class_name))
        dest_dir = os.path.join(stage2_path, class_name)

        # Ensure destination is clean before copying
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
        os.makedirs(dest_dir)

        if src_dir.is_dir():
            print(f"  Copying '{class_name}' to Stage 2 directory...")
            for file_path in src_dir.rglob('*'):
                 if file_path.is_file() and is_valid_image(file_path.name):
                    shutil.copy(file_path, dest_dir)
        else:
            print(f"  ⚠️ Warning: Source directory not found for '{class_name}'")

    print("\n✅ Hierarchical dataset preparation complete.")
    return stage1_path, stage2_path

In [5]:
# -----------------------------------------------
# 3. Utility Functions & Custom Classes
# -----------------------------------------------

# --- Part A: Data Augmentation ---

# 📦 Applies augmentations and processes images on-the-fly for each batch.
# This is a more robust approach than pre-processing the entire dataset.
class DataCollatorWithAugmentation:
    def __init__(self, processor, augment_dict):
        self.processor = processor
        self.augment_dict = augment_dict
        # Baseline augmentation for majority classes.
        self.base_augment = T.Compose([
            T.RandomResizedCrop(size=(224, 224)),
            T.RandomHorizontalFlip(),
            T.RandomRotation(10),
            T.ColorJitter(brightness=0.1, contrast=0.1),
        ])

        # Tensor-domain ops
        self.to_tensor = T.ToTensor()
        self.to_pil = T.ToPILImage()
        self.post_tensor_erase = T.RandomErasing(p=0.2, scale=(0.02, 0.08))

        # Skip erasing on fragile classes
        self.skip_erasing_label_ids = {
            label2id_s2['sadness'],
            label2id_s2['speech_action']
        }
        
    def __call__(self, features):
        processed_images = []
        for x in features:
            label = x["label"]
            rgb_image = x["image"].convert("RGB")

            # 1) apply class-specific PIL pipeline if present; else base PIL pipeline
            pil_aug = self.augment_dict.get(label, self.base_augment)

            img = pil_aug(rgb_image)

            # 2) apply RandomErasing on tensor (only if not skipped)
            if label not in self.skip_erasing_label_ids:
                img_t = self.to_tensor(img)                 # PIL -> Tensor [C,H,W]
                img_t = self.post_tensor_erase(img_t)       # RandomErasing on tensor
                img = self.to_pil(img_t)                    # Tensor -> PIL (processor expects PIL/np)

            processed_images.append(img)

        batch = self.processor(images=processed_images, return_tensors="pt")
        batch["labels"] = torch.tensor([x["label"] for x in features], dtype=torch.long)
        return batch

# --- Part B: Model & Training Components ---

# 🏋️ Defines a custom Trainer that can use either a targeted loss function or class weights.
class CustomLossTrainer(Trainer):
    def __init__(self, *args, loss_fct=None, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = loss_fct
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        if self.loss_fct:
            # Stage 2 uses the custom targeted smoothing loss
            loss = self.loss_fct(logits, labels)
        else:
            # Stage 1 uses standard CrossEntropyLoss with class weights (all on CPU)
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(logits, labels)
            
        return (loss, outputs) if return_outputs else loss


# 🔄 Implements Cross-Entropy Loss with *Targeted* Label Smoothing.
# Smoothing is turned OFF for specified classes to encourage confident predictions. This is used for Stage 2.
class TargetedSmoothedCrossEntropyLoss(nn.Module):
    def __init__(self, smoothing=0.05, target_class_names=None, label2id_map=None, focal_gamma=None):
        super().__init__()
        self.smoothing = smoothing
        self.focal_gamma = focal_gamma  # NEW (None disables focal scaling)
        if target_class_names and label2id_map:
            self.target_class_ids = [label2id_map[name] for name in target_class_names]
        else:
            self.target_class_ids = []

    def forward(self, logits, target):
        num_classes = logits.size(1)
        with torch.no_grad():
            smooth_labels = torch.full_like(logits, self.smoothing / (num_classes - 1))
            smooth_labels.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
            if self.target_class_ids:
                target_mask = torch.isin(target, torch.tensor(self.target_class_ids, device=target.device))
                if target_mask.any():
                    sharp_labels = F.one_hot(target[target_mask], num_classes=num_classes).float()
                    smooth_labels[target_mask] = sharp_labels

        log_probs = F.log_softmax(logits, dim=1)
        ce_per_sample = -(smooth_labels * log_probs).sum(dim=1)

        # NEW: optional focal scaling
        if self.focal_gamma is not None and self.focal_gamma > 0:
            with torch.no_grad():
                probs = torch.softmax(logits, dim=1)
                pt = (probs * smooth_labels).sum(dim=1).clamp_min(1e-6)
            ce_per_sample = ((1 - pt) ** self.focal_gamma) * ce_per_sample

        return ce_per_sample.mean()

# --- Part C: Metrics & Evaluation ---

# 📊 Computes metrics and generates a confusion matrix plot for each evaluation step.
def compute_metrics_with_confusion(eval_pred, label_names, stage_name=""):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    print(f"\n📈 Classification Report for {stage_name}:")
    report = classification_report(labels, preds, target_names=label_names, output_dict=True, zero_division=0)
    print(classification_report(labels, preds, target_names=label_names, zero_division=0))

    # Save raw logits/labels for later analysis like temperature scaling
    np.save(os.path.join(SAVE_DIR, f"logits_eval_{stage_name}_{VERSION}.npy"), logits)
    np.save(os.path.join(SAVE_DIR, f"labels_eval_{stage_name}_{VERSION}.npy"), labels)

    # --- Re-integrated from V28 ---
    # Save per-class F1/precision/recall/entropy to CSV (append per epoch)
    f1s = [report[name]["f1-score"] for name in label_names]
    recalls = [report[name]["recall"] for name in label_names]
    precisions = [report[name]["precision"] for name in label_names]

    # Entropy per class (sorted by entropy)
    softmax_probs = F.softmax(torch.tensor(logits), dim=-1)
    entropies = -torch.sum(softmax_probs * torch.log(softmax_probs + 1e-12), dim=-1)
    entropy_per_class = []
    for idx, class_name in enumerate(label_names):
        mask = (np.array(labels) == idx)
        if mask.any():
            class_entropy = entropies[mask].mean().item()
            entropy_per_class.append((class_name, class_entropy))
        else:
            entropy_per_class.append((class_name, 0.0))
    
    # Create a dictionary for entropies in the correct order for the CSV
    entropy_dict = dict(entropy_per_class)

    # CSV logging
    epoch_metrics_path = os.path.join(SAVE_DIR, f"per_class_metrics_{stage_name}.csv")
    # Access the trainer instance through its global-like availability during compute_metrics call
    active_trainer = trainer_s1 if stage_name == "Stage1" else trainer_s2
    epoch = getattr(active_trainer.state, "epoch", None)

    df_row = pd.DataFrame({
        "epoch": [epoch],
        **{f"f1_{n}": [f] for n, f in zip(label_names, f1s)},
        **{f"recall_{n}": [r] for n, r in zip(label_names, recalls)},
        **{f"precision_{n}": [p] for n, p in zip(label_names, precisions)},
        **{f"entropy_{n}": [entropy_dict[n]] for n in label_names}
    })
    
    if os.path.exists(epoch_metrics_path):
        df_row.to_csv(epoch_metrics_path, mode="a", header=False, index=False)
    else:
        df_row.to_csv(epoch_metrics_path, mode="w", header=True, index=False)
    # --- End Re-integration ---

    # Generate and save a heatmap of the confusion matrix
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_names, yticklabels=label_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix - {stage_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"confusion_matrix_{stage_name}_{VERSION}.png"))
    plt.close()

    # --- Re-integrated from V28 ---
    # Top confused pairs
    confusion_pairs = [
        ((label_names[i], label_names[j]), cm[i][j])
        for i in range(len(label_names))
        for j in range(len(label_names)) if i != j and cm[i][j] > 0
    ]
    top_confusions = sorted(confusion_pairs, key=lambda x: x[1], reverse=True)[:3]
    if top_confusions:
        print("\nTop 3 confused class pairs:")
        for (true_label, pred_label), count in top_confusions:
            print(f"  - {true_label} → {pred_label}: {count} instances")

    # Compute and print entropy metrics
    avg_entropy = entropies.mean().item()
    print(f"\n🧠 Avg prediction entropy: {avg_entropy:.4f}")

    sorted_entropy = sorted(entropy_per_class, key=lambda x: x[1], reverse=True)
    if sorted_entropy:
        print("\n🔍 Class entropies (sorted):")
        for class_name, entropy in sorted_entropy:
            print(f"  - {class_name}: entropy = {entropy:.4f}")
    # --- End Re-integration ---
    
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# --- Part D: Model Saving ---

# 💾 Saves the model and its associated processor to a specified directory.
def save_model_and_processor(model, processor, save_dir, model_name):
    print(f"💾 Saving {model_name} and processor to: {save_dir}")
    model_path = os.path.join(save_dir, model_name)
    os.makedirs(model_path, exist_ok=True)
    model = model.to("cpu")
    processor.save_pretrained(model_path)
    model.save_pretrained(model_path, safe_serialization=True)
    print(f"✅ {model_name} saved successfully.")


# --- Part E: Post-Training Analysis ---

def check_deployment_readiness(metrics_csv_path, f1_threshold=0.80):
    """Analyzes the final metrics CSV to check for production readiness."""
    print("\n" + "="*60)
    print("  DEPLOYMENT READINESS CHECK")
    print("="*60)
    
    if not os.path.exists(metrics_csv_path):
        print(f"⚠️ Metrics file not found at: {metrics_csv_path}")
        return

    metrics_df = pd.read_csv(metrics_csv_path)
    last_epoch_metrics = metrics_df.iloc[-1]
    
    label_names = [col.replace("f1_", "") for col in metrics_df.columns if col.startswith("f1_")]
    
    print(f"Threshold: F1-Score >= {f1_threshold}\n")
    
    issues_found = False
    for label in label_names:
        f1_score = last_epoch_metrics.get(f"f1_{label}", 0)
        if f1_score < f1_threshold:
            print(f"  - ❌ {label:<15} | F1-Score: {f1_score:.2f} (Below Threshold)")
            issues_found = True
        else:
            print(f"  - ✅ {label:<15} | F1-Score: {f1_score:.2f}")
            
    if issues_found:
        print("\n Model is NOT ready for production.")
    else:
        print("\n Model meets the minimum F1-score threshold for all classes.")

In [6]:
# --------------------------
# 4. Main Training Script
# --------------------------

def main(device):
    # Make trainer objects accessible to metrics function
    global trainer_s1, trainer_s2
    
    # --- Sanity Check for Checkpoint Path ---
    if not os.path.exists(PRETRAINED_CHECKPOINT_PATH):
        raise FileNotFoundError(f"Fatal: Pretrained checkpoint not found at {PRETRAINED_CHECKPOINT_PATH}")

    # --- Define specific model paths from the latest checkpoint ---
    s1_checkpoint_path = os.path.join(PRETRAINED_CHECKPOINT_PATH, "relevance_filter_model")
    s2_checkpoint_path = os.path.join(PRETRAINED_CHECKPOINT_PATH, "emotion_classifier_model")

    # The device is now passed in, so the local definition is removed.
    print(f"\n🖥️ Using device: {device}")

    # --- Step 0: Prepare Datasets ---
    # This function copies files into the required two-stage structure.
    # It only needs to be run once.
    prepared_data_path = os.path.join(OUTPUT_ROOT_DIR, "prepared_datasets")
    if PREPARE_DATASETS:
        stage1_dataset_path, stage2_dataset_path = prepare_hierarchical_datasets(BASE_DATASET_PATH, prepared_data_path)
    else:
        stage1_dataset_path = os.path.join(prepared_data_path, "stage_1_relevance_dataset")
        stage2_dataset_path = os.path.join(prepared_data_path, "stage_2_emotion_dataset")
        print("✅ Skipping dataset preparation, using existing directories.")
    
    # # --- Set hardware device ---
    # # commented out due to present mps and pytorch incompatibilities
    # device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # print(f"\n🖥️ Using device: {device}")

    # ==========================================================================
    #   STAGE 1: TRAIN RELEVANCE FILTER (BINARY CLASSIFIER)
    # ==========================================================================
    print("\n" + "="*60)
    print("  STAGE 1: TRAINING RELEVANCE FILTER (BINARY CLASSIFIER)")
    print("="*60)

    # --- Load Stage 1 data ---
    stage1_output_dir = os.path.join(SAVE_DIR, "stage_1_relevance_model_training")
    dataset_s1 = load_dataset("imagefolder", data_dir=stage1_dataset_path, split='train').train_test_split(test_size=0.2, seed=42)
    train_dataset_s1 = dataset_s1["train"]
    eval_dataset_s1 = dataset_s1["test"]
    print(f"Stage 1: {len(train_dataset_s1)} training samples, {len(eval_dataset_s1)} validation samples.")

    # --- Configure Stage 1 model ---
    # We load the base processor once.
    processor = AutoImageProcessor.from_pretrained(BASE_MODEL_NAME)
    # Load the pretrained checkpoint but replace the final layer (classifier head)
    # for our binary (2-label) task.
    model_s1 = ViTForImageClassification.from_pretrained(
        s1_checkpoint_path, # <-- Use the specific path for the Stage 1 model
        num_labels=2,
        label2id=label2id_s1,
        id2label=id2label_s1,
        ignore_mismatched_sizes=True
    ).to(device)

    # --- Handle Extreme Class Imbalance in Stage 1 with Class Weights ---
    # This is critical because the 'irrelevant' class is much larger than the 'relevant' class.
    class_weights_s1 = compute_class_weight('balanced', classes=np.unique(train_dataset_s1['label']), y=train_dataset_s1['label'])
    class_weights_s1 = torch.tensor(class_weights_s1, dtype=torch.float).to(device)
    print(f"⚖️ Stage 1 Class Weights: {class_weights_s1}")

    # --- Define Early Stopping ---
    # Stops training if validation loss doesn't improve for 2 consecutive epochs
    early_stop_callback = EarlyStoppingCallback(
        early_stopping_patience=2,
        early_stopping_threshold=0.001
    )
    
    # --- Set up Stage 1 Trainer ---
    training_args_s1 = TrainingArguments(
        output_dir=stage1_output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        use_cpu=True,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        logging_dir=os.path.join(stage1_output_dir, "logs"),
        logging_strategy="steps",
        logging_steps=50,
        remove_unused_columns=False,
    )

    # --- Set up Stage 1 Trainer ---
    # The complex discriminative learning rate and layer freezing strategy in 
        # V31 caused a severe performance drop. This change reverts Stage 1 to 
        # V30's simpler and more effective approach of using a single, uniform 
        # learning rate for the entire model, which is managed by the Hugging 
        # Face Trainer's default optimizer.
    training_args_s1.learning_rate = 3e-5 # Set learning rate directly
    
    # Use the flexible CustomLossTrainer, passing the class weights to it.
    trainer_s1 = CustomLossTrainer(
        model=model_s1,
        args=training_args_s1,
        train_dataset=train_dataset_s1,
        eval_dataset=eval_dataset_s1,
        compute_metrics=partial(compute_metrics_with_confusion, label_names=list(id2label_s1.values()), stage_name="Stage1"),
        data_collator=DataCollatorWithAugmentation(processor=processor, augment_dict={}), # Use base augmentation for all
        class_weights=class_weights_s1, # Pass weights to the trainer
        callbacks=[early_stop_callback] # Keep early stopping
    )

    # --- Train Stage 1 model ---
    print("🚀 Starting Stage 1 training...")
    start_time_s1 = time.time() # Record start time
    trainer_s1.train()
    end_time_s1 = time.time()   # Record end time
    
    # Calculate and print the duration
    duration_s1 = end_time_s1 - start_time_s1
    print(f"⌛ Stage 1 training took: {time.strftime('%H:%M:%S', time.gmtime(duration_s1))}")
    save_model_and_processor(trainer_s1.model, processor, SAVE_DIR, model_name="relevance_filter_model")
    print("\n✅ Stage 1 Training Complete.")

    # ==========================================================================
    #   STAGE 2: TRAIN EMOTION CLASSIFIER (11-CLASS)
    # ==========================================================================
    print("\n" + "="*60)
    print(f"  STAGE 2: TRAINING EMOTION CLASSIFIER ({len(RELEVANT_CLASSES)}-CLASS)")
    print("="*60)

    # --- Load Stage 2 data ---
    stage2_output_dir = os.path.join(SAVE_DIR, "stage_2_emotion_model_training")
    dataset_s2 = load_dataset("imagefolder", data_dir=stage2_dataset_path, split='train').train_test_split(test_size=0.2, seed=42)
    train_dataset_s2 = dataset_s2["train"]
    eval_dataset_s2 = dataset_s2["test"]
    print(f"Stage 2: {len(train_dataset_s2)} training samples, {len(eval_dataset_s2)} validation samples.")
    print("Stage 2 Label Distribution (Train):", Counter(train_dataset_s2['label']))


    # --- Configure Stage 2 model ---
    # Load the pretrained checkpoint again, this time with a classifier head for our 11 emotion classes.
    model_s2 = ViTForImageClassification.from_pretrained(
        s2_checkpoint_path, # <-- Use the specific path for the Stage 2 model
        num_labels=len(RELEVANT_CLASSES),
        label2id=label2id_s2,
        id2label=id2label_s2,
        ignore_mismatched_sizes=True
    ).to(device)

    # --- Define Augmentation and Loss for Stage 2 ---
    # Apply stronger augmentation to the minority classes to help the model learn them better.
    minority_aug = T.Compose([
        RandAugment(num_ops=2, magnitude=9),
        T.RandomResizedCrop(224, scale=(0.7, 1.0)),
        T.ColorJitter(0.3, 0.3, 0.3, 0.1),
    ])

    # The addition of 'sadness' and 'speech_action' to the heavy augmentation pipeline in V31 
        # was counterproductive, causing the F1-scores for these classes to collapse. 
        # This change reverts the list to the V30 definition, removing the aggressive 
        # augmentation from the classes it harmed.
    minority_classes_s2 = [label2id_s2[name] for name in ['disgust', 'questioning', 'contempt', 'fear']]
    minority_augment_map_s2 = {label_id: minority_aug for label_id in minority_classes_s2}

    # NEW: very mild, targeted aug ONLY for the weakest classes (no RandAugment)
    mild_aug = T.Compose([
        T.RandomResizedCrop(224, scale=(0.9, 1.0)),
        T.RandomHorizontalFlip(),
        T.ColorJitter(0.05, 0.05, 0.05, 0.02),
        T.RandomPerspective(distortion_scale=0.05, p=0.3),
    ])
    targeted_mild_classes = [label2id_s2['sadness'], label2id_s2['speech_action']]
    targeted_mild_map_s2 = {label_id: mild_aug for label_id in targeted_mild_classes}

    # MERGE: single mapping passed to the collator (class id -> transform)
    augment_dict = {**minority_augment_map_s2, **targeted_mild_map_s2}

    # Use the custom loss function to turn off label smoothing for historically difficult classes.
        # Turn OFF smoothing for the hardest classes (sharper targets) and apply mild focal emphasis
    loss_fct_s2 = TargetedSmoothedCrossEntropyLoss(
        smoothing=0.05,
        target_class_names=['sadness', 'speech_action'],   
        label2id_map=label2id_s2,
        focal_gamma=1.5                                    
    )

    # --- Set up Stage 2 Trainer ---
    # Adding weight decay, cosine scheduler + warmup, grad accumulation improves stability 
        # (especially on CPU/small batch) without altering your high-level flow.
    training_args_s2 = TrainingArguments(
        output_dir=stage2_output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        use_cpu=True,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=6,                       # +1 epoch for minorities
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        logging_dir=os.path.join(stage2_output_dir, "logs"),
        logging_strategy="epoch",
        remove_unused_columns=False,
        weight_decay=0.05,                        # NEW
        lr_scheduler_type="cosine",               # NEW
        warmup_ratio=0.10,                        # NEW
        gradient_accumulation_steps=2,            # NEW
    )
    training_args_s2.learning_rate = 4e-5


    # --- Set up Stage 2 Trainer ---
    # As with Stage 1, the complex fine-tuning strategy implemented in V31 failed. 
        # This change reverts the Stage 2 training process to V30's more effective 
        # uniform learning rate strategy to restore model performance.
    training_args_s2.learning_rate = 4e-5 # Set learning rate directly

    # Use the CustomLossTrainer again, passing the targeted loss function.
    trainer_s2 = CustomLossTrainer(
        model=model_s2,
        args=training_args_s2,
        train_dataset=train_dataset_s2,
        eval_dataset=eval_dataset_s2,
        compute_metrics=partial(compute_metrics_with_confusion, label_names=RELEVANT_CLASSES, stage_name="Stage2"),
        data_collator=DataCollatorWithAugmentation(processor=processor, augment_dict=augment_dict),
        loss_fct=loss_fct_s2, # Pass custom loss function
        callbacks=[early_stop_callback] # Keep early stopping
    )

    # --- Train Stage 2 model ---
    print("🚀 Starting Stage 2 training...")
    start_time_s2 = time.time() # Record start time
    trainer_s2.train()
    end_time_s2 = time.time()   # Record end time
    
    # Calculate and print the duration
    duration_s2 = end_time_s2 - start_time_s2
    print(f"⌛ Stage 2 training took: {time.strftime('%H:%M:%S', time.gmtime(duration_s2))}")
    save_model_and_processor(trainer_s2.model, processor, SAVE_DIR, model_name="emotion_classifier_model")
    print("\n✅ Stage 2 Training Complete.")
    print("\n🎉 Hierarchical Training Pipeline Finished Successfully.")
    
    # Return the trained models and processor to be used by analysis functions
    return trainer_s1.model, trainer_s2.model, processor

In [7]:
# ----------------------------------
# 5. Hierarchical Inference
# ----------------------------------
# This function defines the two-step prediction pipeline for new images.
# It first checks for relevance (Stage 1) and then classifies the emotion (Stage 2).

def hierarchical_predict(image_paths, model_s1, model_s2, processor, device, batch_size=32):
    results = []
    for i in tqdm(range(0, len(image_paths), batch_size), desc="🔬 Running Hierarchical Inference"):
        batch_paths = image_paths[i:i+batch_size]
        images = []
        valid_paths = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
                valid_paths.append(path)
            except Exception:
                continue

        if not images:
            continue

        inputs = processor(images=images, return_tensors="pt").to(device)
        
        # --- Stage 1 Prediction: Is the image relevant? ---
        with torch.no_grad():
            logits_s1 = model_s1(**inputs).logits
            probs_s1 = F.softmax(logits_s1, dim=-1)
        
        # Calibrated threshold for 'relevant' (label id = 1)
        tau = 0.45
        # Create a mask of images that were classified as 'relevant'
        relevant_mask = (probs_s1[:, label2id_s1['relevant']] >= tau)
        preds_s1 = torch.where(relevant_mask, torch.tensor(label2id_s1['relevant']), torch.tensor(label2id_s1['irrelevant']))


        # --- Stage 2 Prediction (only on relevant images) ---
        if relevant_mask.any():
            # Filter the input tensors to only include the relevant images
            relevant_inputs = {k: v[relevant_mask] for k, v in inputs.items()}

            with torch.no_grad():
                logits_s2 = model_s2(**relevant_inputs).logits
                probs_s2 = F.softmax(logits_s2, dim=-1)
                confs_s2, preds_s2 = torch.max(probs_s2, dim=-1)

        # --- Aggregate Results ---
        # Loop through the original batch and assign the correct prediction
        s2_idx = 0
        for j in range(len(valid_paths)):
            if relevant_mask[j]:
                # If relevant, get the prediction from the Stage 2 model
                pred_label = id2label_s2[preds_s2[s2_idx].item()]
                confidence = confs_s2[s2_idx].item()
                s2_idx += 1
            else:
                # If not relevant, label it and stop
                pred_label = "irrelevant"
                confidence = torch.softmax(logits_s1[j], dim=-1)[preds_s1[j]].item()

            results.append({
                "image_path": valid_paths[j],
                "prediction": pred_label,
                "confidence": confidence
            })
    return results

In [8]:
# ==============================================================================
# 6. Post-Training Analysis, Review, and Curation
# ==============================================================================

def run_post_training_analysis(model_s1, model_s2, processor, device, base_dataset_path, save_dir, version):
    """
    Runs a full inference pass and generates logs for review, curation, and analysis.
    Combines logic from old sections 15 and 16.
    """
    print("\n" + "="*60)
    print("  RUNNING POST-TRAINING ANALYSIS & CURATION WORKFLOW")
    print("="*60)

    # --- Part A: Run Hierarchical Inference on the Entire Dataset ---
    all_image_paths = [str(p) for p in Path(base_dataset_path).rglob("*") if is_valid_image(p.name)]
    print(f"Found {len(all_image_paths)} images to process for inference.")
    
    predictions = hierarchical_predict(all_image_paths, model_s1, model_s2, processor, device)
    df = pd.DataFrame(predictions)
    
    # Derive true label from path for analysis
    df['true_label'] = df['image_path'].apply(lambda p: Path(p).parent.name)

    # Save the full log
    full_log_path = os.path.join(save_dir, f"{version}_full_inference_log.csv")
    df.to_csv(full_log_path, index=False)
    print(f"\n✅ Full inference log saved to: {full_log_path}")

    # --- Part B: Identify and Organize Images for Manual Review ---
    # Tag images with low confidence as "REVIEW"
    review_threshold = 0.85
    review_df = df[df['confidence'] < review_threshold]
    
    review_sort_dir = os.path.join(save_dir, "review_candidates_by_predicted_class")
    os.makedirs(review_sort_dir, exist_ok=True)
    
    print(f"\nFound {len(review_df)} images below {review_threshold} confidence for review.")
    for _, row in tqdm(review_df.iterrows(), total=len(review_df), desc="Sorting review images"):
        dest_dir = os.path.join(review_sort_dir, row['prediction'])
        os.makedirs(dest_dir, exist_ok=True)
        shutil.copy(row['image_path'], dest_dir)
    print(f"📂 Sorted review images into folders at: {review_sort_dir}")

    # --- Part C: Mine for "Hard Negative" Confusion Pairs ---
    # Find images where the model was confused between specific, problematic classes
    confusion_pairs_to_mine = [('contempt', 'questioning'), ('contempt', 'neutral'), ('fear', 'surprise')]
    
    print("\n⛏️  Mining for hard negative confusion pairs...")
    for pair in confusion_pairs_to_mine:
        c1, c2 = pair
        # Find images where true is c1 but predicted is c2, OR true is c2 and predicted is c1
        mask = ((df['true_label'] == c1) & (df['prediction'] == c2)) | \
               ((df['true_label'] == c2) & (df['prediction'] == c1))
        
        hard_negatives = df[mask]
        
        if not hard_negatives.empty:
            out_path = os.path.join(save_dir, f"hard_negatives_{c1}_vs_{c2}.csv")
            hard_negatives.to_csv(out_path, index=False)
            print(f"  - Found {len(hard_negatives)} hard negatives for {pair}. Saved to: {out_path}")

In [9]:
# ==============================================================================
# 7. Model Calibration
# ==============================================================================

def apply_temperature_scaling(logits, labels):
    """Finds the optimal temperature for calibrating model confidence."""
    logits_tensor = torch.tensor(logits, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    class TemperatureScaler(nn.Module):
        def __init__(self):
            super().__init__()
            self.temperature = nn.Parameter(torch.ones(1) * 1.5)

        def forward(self, logits):
            return logits / self.temperature

    model = TemperatureScaler()
    optimizer = LBFGS([model.temperature], lr=0.01, max_iter=50)

    def eval_fn():
        optimizer.zero_grad()
        loss = F.cross_entropy(model(logits_tensor), labels_tensor)
        loss.backward()
        return loss

    optimizer.step(eval_fn)
    return model.temperature.item()

def plot_reliability_diagram(logits, labels, temperature, save_dir, version, stage_name):
    """Visualizes model calibration before and after temperature scaling."""
    logits = torch.from_numpy(logits)
    labels = torch.from_numpy(labels)
    
    # Calculate before
    probs_before = F.softmax(logits, dim=1)
    confs_before, _ = torch.max(probs_before, 1)
    
    # Calculate after
    probs_after = F.softmax(logits / temperature, dim=1)
    confs_after, _ = torch.max(probs_after, 1)

    # Plotting logic remains the same...
    # (For brevity, the detailed plotting code from your old script goes here)
    print(f"📊 Reliability diagram generation logic would go here.")

In [10]:
# ==============================================================================
# 8. Hierarchical Model Ensembling
# ==============================================================================

def hierarchical_ensemble_predict(image_path, processor, s1_models, s2_models, device):
    """Performs an ensembled prediction using multiple hierarchical models."""
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

    except Exception:
        return None, None

    # --- Stage 1 Ensemble (Majority Vote) ---
    s1_votes = []
    with torch.no_grad():
        for model in s1_models:
            logits = model(**inputs).logits
            pred = torch.argmax(logits, dim=-1).item()
            s1_votes.append(pred)
    
    # Decide relevance based on majority vote (1 = relevant)
    is_relevant = Counter(s1_votes).most_common(1)[0][0] == label2id_s1['relevant']

    if not is_relevant:
        return "irrelevant", None

    # --- Stage 2 Ensemble (Average Probabilities) ---
    s2_probs = []
    with torch.no_grad():
        for model in s2_models:
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)
            s2_probs.append(probs)
            
    # Average the probabilities across all models
    avg_probs = torch.mean(torch.stack(s2_probs), dim=0)
    confidence, pred_idx = torch.max(avg_probs, dim=-1)
    
    final_prediction = id2label_s2[pred_idx.item()]
    final_confidence = confidence.item()
    
    return final_prediction, final_confidence

In [11]:
# ==============================================================================
# 9. Script Execution Entry Point
# ==============================================================================
if __name__ == "__main__":

    # Define the device once for the entire script run.
    device = torch.device("cpu")
    
    # --- Step 1: Execute Training Pipeline ---
    # The main function now returns the trained models and processor
    model_s1, model_s2, processor = main(device)
    
    # --- Step 2: Run Post-Training Analysis & Curation ---
    if RUN_INFERENCE:
        # This function runs the full inference pass and generates logs for review.
        # It uses the in-memory models returned from main().
        run_post_training_analysis(model_s1, model_s2, processor, device, BASE_DATASET_PATH, SAVE_DIR, VERSION)
    
    # --- Step 3: Run Final Model Checks ---
    # Check if the model is ready for "deployment" based on F1 scores
    stage2_metrics_path = os.path.join(SAVE_DIR, "per_class_metrics_Stage2.csv")
    check_deployment_readiness(stage2_metrics_path, f1_threshold=0.80)
    
    # --- Step 4: Calibrate the Stage 2 Model ---
    logits_s2_path = os.path.join(SAVE_DIR, f"logits_eval_Stage2_{VERSION}.npy")
    labels_s2_path = os.path.join(SAVE_DIR, f"labels_eval_Stage2_{VERSION}.npy")
    
    if os.path.exists(logits_s2_path) and os.path.exists(labels_s2_path):
        print("\n" + "="*60)
        print("  CALIBRATING STAGE 2 MODEL")
        print("="*60)
        logits_s2 = np.load(logits_s2_path)
        labels_s2 = np.load(labels_s2_path)
        
        optimal_temp = apply_temperature_scaling(logits_s2, labels_s2)
        print(f"✅ Optimal temperature for Stage 2 model: {optimal_temp:.4f}")
        # plot_reliability_diagram(logits_s2, labels_s2, optimal_temp, SAVE_DIR, VERSION, "Stage2")
    else:
        print("⚠️ Skipping calibration, logits/labels files for Stage 2 not found.")

    # COME BACK LATER TO MAKE DYNAMIC AND AUTOMATED LOADING OF PATH
    # --- Step 5: (Hypothetical) Run Ensemble Analysis ---
    # Use the saved V32 artifacts as the "previous" models for ensembling
    v_prev_path = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V32_20251008_115114"
    
    if os.path.exists(v_prev_path):
        print("\n" + "="*60)
        print("  RUNNING HIERARCHICAL ENSEMBLE ANALYSIS (current + V32)")
        print("="*60)
        
        # Load the older V32 models for the ensemble
        s1_model_prev = AutoModelForImageClassification.from_pretrained(
            os.path.join(v_prev_path, "relevance_filter_model")
        ).to(device).eval()
        s2_model_prev = AutoModelForImageClassification.from_pretrained(
            os.path.join(v_prev_path, "emotion_classifier_model")
        ).to(device).eval()
        
        # Use the in-memory models from THIS run (e.g., V33 when you launch it)
        # Assumes you have model_s1 and model_s2 already defined in memory
        s1_models_ensemble = [model_s1, s1_model_prev]
        s2_models_ensemble = [model_s2, s2_model_prev]

        # NEW: auto-pick a real image from ANY non-empty predicted-class folder
        review_root = os.path.join(v_prev_path, "review_candidates_by_predicted_class")
        example_image_path = None
        if os.path.isdir(review_root):
            for cls in os.listdir(review_root):
                cls_dir = os.path.join(review_root, cls)
                if os.path.isdir(cls_dir):
                    imgs = [f for f in os.listdir(cls_dir) if f.lower().endswith((".jpg",".jpeg",".png",".tif",".tiff"))]
                    if imgs:
                        example_image_path = os.path.join(cls_dir, imgs[0])
                        break
    
        if example_image_path and os.path.exists(example_image_path):
            prediction, confidence = hierarchical_ensemble_predict(
                example_image_path, processor, s1_models_ensemble, s2_models_ensemble, device
            )
            print(f"Ensemble prediction for {Path(example_image_path).name}: {prediction} (Confidence: {confidence:.2f})")
        else:
            print("ℹ️ Skipping ensemble demo: no example image found under 'review_candidates_by_predicted_class'.")


🖥️ Using device: cpu
✅ Skipping dataset preparation, using existing directories.

  STAGE 1: TRAINING RELEVANCE FILTER (BINARY CLASSIFIER)


Resolving data files:   0%|          | 0/26881 [00:00<?, ?it/s]

Stage 1: 21504 training samples, 5377 validation samples.


/Users/natalyagrokh/miniconda3/envs/ml_expressions_v5/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


⚖️ Stage 1 Class Weights: tensor([0.6492, 2.1761])
🚀 Starting Stage 1 training...


...


📈 Classification Report for Stage1:
              precision    recall  f1-score   support

  irrelevant       0.91      0.75      0.82      4132
    relevant       0.47      0.75      0.58      1245

    accuracy                           0.75      5377
   macro avg       0.69      0.75      0.70      5377
weighted avg       0.81      0.75      0.76      5377


Top 3 confused class pairs:
  - irrelevant → relevant: 1053 instances
  - relevant → irrelevant: 307 instances

🧠 Avg prediction entropy: 0.5260

🔍 Class entropies (sorted):
  - irrelevant: entropy = 0.5280
  - relevant: entropy = 0.5195

📈 Classification Report for Stage1:
              precision    recall  f1-score   support

  irrelevant       0.89      0.86      0.87      4132
    relevant       0.57      0.63      0.60      1245

    accuracy                           0.81      5377
   macro avg       0.73      0.74      0.74      5377
weighted avg       0.81      0.81      0.81      5377


Top 3 confused class pairs:
  - 

Resolving data files:   0%|          | 0/6175 [00:00<?, ?it/s]

Stage 2: 4940 training samples, 1235 validation samples.
Stage 2 Label Distribution (Train): Counter({9: 1608, 4: 651, 8: 554, 5: 530, 0: 388, 6: 382, 1: 251, 3: 240, 10: 135, 7: 101, 2: 100})
🚀 Starting Stage 2 training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.554300,0.442784,0.801619
2,0.403600,0.338347,0.830769
3,0.318900,0.338208,0.844534
4,0.266700,0.291141,0.855870
5,0.230000,0.281384,0.859109
6,0.219200,0.280947,0.863158



📈 Classification Report for Stage2:
                precision    recall  f1-score   support

         anger       0.79      0.71      0.75        85
      contempt       0.72      0.83      0.78        60
       disgust       0.86      0.73      0.79        26
          fear       0.94      0.90      0.92        71
     happiness       0.86      0.81      0.83       167
       neutral       0.80      0.73      0.76       135
   questioning       0.73      0.68      0.71        92
       sadness       1.00      0.30      0.46        40
      surprise       0.74      0.84      0.78       147
neutral_speech       0.80      0.89      0.84       381
 speech_action       0.84      0.84      0.84        31

      accuracy                           0.80      1235
     macro avg       0.83      0.75      0.77      1235
  weighted avg       0.81      0.80      0.80      1235


Top 3 confused class pairs:
  - neutral → neutral_speech: 23 instances
  - sadness → neutral_speech: 18 instances
  - h

In [12]:
import numpy as np, csv

labels = np.load(os.path.join(SAVE_DIR, "labels_eval_Stage2_V33.npy"))
logits = np.load(os.path.join(SAVE_DIR, "logits_eval_Stage2_V33.npy"))
probs = np.exp(logits - logits.max(axis=1, keepdims=True))
probs = probs / probs.sum(axis=1, keepdims=True)
pred = probs.argmax(axis=1)
conf = probs.max(axis=1)

out_csv = os.path.join(SAVE_DIR, "v33_eval_mismatches.csv")
with open(out_csv, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["index", "true_label", "predicted_label", "confidence"])
    for i, (t, p, c) in enumerate(zip(labels, pred, conf)):
        if t != p:
            w.writerow([i, id2label_s2[int(t)], id2label_s2[int(p)], float(c)])

print(f"✅ Wrote eval mismatches (indices only): {out_csv}")

✅ Wrote eval mismatches (indices only): /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V33_20251010_122749/v33_eval_mismatches.csv


In [16]:
import os, glob, torch, pandas as pd, numpy as np
from datasets import load_dataset
from transformers import AutoImageProcessor, ViTForImageClassification
from tqdm import tqdm
from PIL import Image

# ---------------------------
# Config (auto-detect latest V33 folder)
# ---------------------------
ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training"
candidates = sorted(glob.glob(os.path.join(ROOT, "V33_*")), key=os.path.getmtime, reverse=True)
SAVE_DIR = candidates[0] if candidates else "/ABSOLUTE/PATH/TO/V33_YYYYMMDD_HHMMSS"
print(f"Using SAVE_DIR = {SAVE_DIR}")

BASE_MODEL_NAME = "google/vit-base-patch16-224-in21k"
prepared_data_path = os.path.join(ROOT, "prepared_datasets")
stage2_dataset_path = os.path.join(prepared_data_path, "stage_2_emotion_dataset")
device = torch.device("cpu")  # keep CPU for stability

RELEVANT_CLASSES = [
    'neutral', 'neutral_speech', 'happiness', 'sadness', 'anger',
    'fear', 'surprise', 'disgust', 'contempt', 'questioning', 'speech_action'
]
label2id_s2 = {n:i for i,n in enumerate(RELEVANT_CLASSES)}
id2label_s2 = {i:n for n,i in label2id_s2.items()}

# ---------------------------
# Helpers
# ---------------------------
def ensure_rgb(img):
    """
    Normalize any image to a proper RGB PIL.Image.
    Handles grayscale (H,W), channel-first (C,H,W), and palette images.
    """
    if isinstance(img, Image.Image):
        # Convert any PIL mode ('L','P','LA','RGBA', etc.) to 'RGB'
        return img.convert("RGB")
    # If it's a numpy array
    arr = np.array(img)
    if arr.ndim == 2:  # grayscale HxW
        arr = np.stack([arr]*3, axis=-1)
    elif arr.ndim == 3:
        # If channel-first (C,H,W), move to (H,W,C)
        if arr.shape[0] in (1, 3) and arr.shape[-1] not in (1, 3):
            arr = np.transpose(arr, (1, 2, 0))
        # If single-channel HxWx1, tile to 3 channels
        if arr.shape[-1] == 1:
            arr = np.repeat(arr, 3, axis=-1)
        # If more than 3 channels (e.g., RGBA), drop alpha
        if arr.shape[-1] > 3:
            arr = arr[..., :3]
    else:
        raise ValueError(f"Unsupported image ndim: {arr.ndim}")
    # Ensure uint8 range for PIL
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    return Image.fromarray(arr, mode="RGB")

# ---------------------------
# Rehydrate model + eval set
# ---------------------------
s2_checkpoint_path = os.path.join(SAVE_DIR, "emotion_classifier_model")
processor = AutoImageProcessor.from_pretrained(BASE_MODEL_NAME)
model_s2 = ViTForImageClassification.from_pretrained(
    s2_checkpoint_path,
    num_labels=len(RELEVANT_CLASSES),
    label2id=label2id_s2,
    id2label=id2label_s2
).to(device).eval()

dataset_s2 = load_dataset("imagefolder", data_dir=stage2_dataset_path, split="train").train_test_split(test_size=0.2, seed=42)
eval_dataset_s2 = dataset_s2["test"]
print(f"✅ Eval dataset ready: {len(eval_dataset_s2)} samples")

# ---------------------------
# Eval-only inference → CSV
# ---------------------------
rows = []
for ex in tqdm(eval_dataset_s2, desc="Evaluating"):
    raw_img = ex["image"]
    img = ensure_rgb(raw_img)  # <-- critical fix
    true_id = int(ex["label"])
    true_label = id2label_s2[true_id]

    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model_s2(**inputs).logits
        probs = torch.softmax(logits, dim=1).squeeze(0)
        pred_id = int(torch.argmax(probs).item())
        conf = float(probs[pred_id].item())

    rows.append({
        "filepath": getattr(raw_img, "filename", "N/A"),
        "true_label": true_label,
        "predicted_label": id2label_s2[pred_id],
        "confidence": round(conf, 4)
    })

df_results = pd.DataFrame(rows)
out_csv = os.path.join(SAVE_DIR, "V33_full_inference_log.csv")
df_results.to_csv(out_csv, index=False)
print(f"✅ Inference complete: wrote {len(df_results)} rows to {out_csv}")

# ---------------------------
# Mini confusion summary
# ---------------------------
summary = df_results.groupby(["true_label", "predicted_label"]).size().unstack(fill_value=0)
print("\n📊 Quick Confusion Summary:")
print(summary)


Using SAVE_DIR = /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V33_20251010_122749


Resolving data files:   0%|          | 0/6175 [00:00<?, ?it/s]

✅ Eval dataset ready: 1235 samples


Evaluating: 100%|███████████████████████████| 1235/1235 [01:11<00:00, 17.38it/s]

✅ Inference complete: wrote 1235 rows to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V33_20251010_122749/V33_full_inference_log.csv

📊 Quick Confusion Summary:
predicted_label  anger  contempt  disgust  fear  happiness  neutral  \
true_label                                                            
anger              163         0        0     0          0        0   
contempt             0       140        0     0          0        3   
disgust              0         0       31     1          0        1   
fear                 0         0        0   110          0        0   
happiness            0         0        0     0         25        0   
neutral              0         0        1     0          1       73   
neutral_speech       1         0        1     5          3       10   
questioning         10         1       27     3          0        7   
sadness              0         0        0     0          0        0   
speech_action        0         0    